# Using Clipped PPO for Automated Model Compression


Continuing on the work of [AMC](https://arxiv.org/abs/1802.03494) we replace DDPG with Clipped PPO.

Results are interesting and encouraging as there is learning.  However, this is less sample-efficient compared to DDPG, and therefore takes longer.

We search for a 50%-MACs-constrained (FLOPs-constrained) Plain20.  From Greedy Search algorithm we know that there exists a 50%-MACs-constrained Plain20 that can provide Top1=90%.  The current fine-tuned Plain20 model from our PPO experiments has a Top1=89%.

## Experiment setup


### Clipped PPO configuration

### Distiller Clipped PPO AMC experiments


## Notebook code

Skip this part - it is necessary only for creating the diagrams.  You may also toggle the code-view button.

In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off code view"></form>''')

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib 
import csv
from matplotlib.ticker import FuncFormatter
import ipywidgets as widgets
from ipywidgets import interactive, interact, Layout
import matplotlib.pylab as pylab
import matplotlib.animation as animation
from matplotlib import animation, rc



#plt.style.use('seaborn') # pretty matplotlib plots

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 7),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'xx-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)


def to_percent(y, position):
    # Ignore the passed in position. This has the effect of scaling the default
    # tick locations.
    if y < 1:
        y = 100 * y
    s = "{:.1f}".format(y)

    # The percent symbol needs escaping in latex
    if matplotlib.rcParams['text.usetex'] is True:
        return s + r'$\%$'
    else:
        return s + '%'
    
# Widen the cells to get entire rows in the screen.
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

import json

def plot_layer_compute_densities(df, idx, ax=None, color=None):
    if ax is None:
        plt.figure()
        ax = plt
    
    record = df.iloc[idx]
    net_performance = json.loads(record["performance"])
    ax.bar(range(len(net_performance)), list(net_performance.values()), color=color, align='center')
    ax.set_title("Ep:{} - Top1:{:.1f}%\nMACs:{:.1f}%".format(record['episode'], 
                                                             record['top1'], 
                                                             record['normalized_macs']))
    #ax.set_xticks(range(len(net_performance)), list(net_performance.keys()))


def plot_action_history(df, idx, action_type='action_history', ax=None, color=None):
    if ax is None:
        plt.figure()
        ax = plt
    
    record = df.iloc[idx]
    layer_sparsities = json.loads(record[action_type])
    #layer_sparsities = record[action_type]
    #layer_sparsities = layer_sparsities[1:-1].split(",")
    layer_densities = [1.- float(sparsity) for sparsity in layer_sparsities]
    ax.bar(range(len(layer_densities)), layer_densities, color=color)
    ax.set_title("Ep:{} - Top1:{:.1f}%\nMACs:{:.1f}%".format(record['episode'], 
                                                             record['top1'], 
                                                             record['normalized_macs']))



def smooth(data, win_size):
    if not win_size:
        return data
    win_size = max(0, win_size)
    return [np.mean(data[max(0, i-win_size):i]) for i in range(len(data))]


def plot_performance(title, dfs, alpha, window_size, top1, macs, params, reward, start=0, end=-1, plot_type='error'):
    plot_kwargs = {"figsize":(15,7), "lw": 1, "alpha": alpha, "title": title, "grid": True}
    smooth_kwargs = {"lw": 2 if window_size > 0 else 1, "legend": True, "grid": True}
    
    if not isinstance(dfs, list):
        dfs = [dfs]
    # Apply zoom
    df_end = min([len(df) for df in dfs])
    if end > 0:
        #df_end = min(df_end, end)
        end = min(df_end, end)
    else:
        end = df_end
    print(end)
    #dfs = [df[:df_end].copy() for df in dfs] 
    dfs = [df for df in dfs] 
    df = dfs[0]
    left_axs, right_axs = [], []
    
    if macs:
        ax = df['normalized_macs'][start:end].plot(**plot_kwargs, color="r")
        left_axs.append((ax, "MACs"))
        #ax.set(xlabel="Episode", ylabel="(%)")
        #ax.set_ylim([0,100])
        df['smooth_normalized_macs'] = smooth(df['normalized_macs'], window_size)
        df['smooth_normalized_macs'][start:end].plot(**smooth_kwargs, color="r")

    if top1:
        for df in dfs:
            df['smooth_top1'] = smooth(df['top1'], window_size)
        
        if len(dfs) > 1:
            plot_kwargs['alpha'] = 1.0
            plot_kwargs['legend'] = True
            if plot_type == 'error':
                top1_len = min([len(df) for df in dfs])
                dfs = [df[:top1_len] for df in dfs]
                dfs_top1 = [df['smooth_top1'] for df in dfs]
                dfs_top1_dp = pd.DataFrame(dfs_top1)
                top1_min = dfs_top1_dp.min(axis=0)
                top1_max = dfs_top1_dp.max(axis=0)
                top1_mean = dfs_top1_dp.mean(axis=0)
                
                display_mean = False
                if display_mean:
                    ax = top1_mean.plot(**plot_kwargs, color="b")
                
                for p in dfs_top1:
                    ax = p[start:end].plot(**plot_kwargs)
                if display_mean:
                    ax.legend(['mean'] + [str(i+1) for i in range(len(dfs_top1))])
                else:
                    ax.legend([str(i+1) for i in range(len(dfs_top1))])
                ax.set(xlabel="Episode", ylabel="(%)")            
                ax.fill_between(range(len(top1_min)), top1_max, top1_min, color="b", alpha=0.3)

                left_axs.append((ax, "Top1"))
            else:
                assert plot_type == 'compare'
                dfs_top1 = [df['smooth_top1'] for df in dfs]
                for p in dfs_top1:
                    #ax = p[start:end].plot(**plot_kwargs)
                    ax = p[start:].plot(**plot_kwargs)
                ax.legend([str(i+1) for i in range(len(dfs_top1))])
                left_axs.append((ax, "Top1"))
        else:
            ax = df['top1'][start:end].plot(**plot_kwargs, color="b")
            left_axs.append((ax, "Top1"))
            #ax.set(xlabel="Episode", ylabel="(%)")
            df['smooth_top1'][start:end].plot(**smooth_kwargs, color="b")
            
    if params:
        ax = df['normalized_nnz'][start:end].plot(**plot_kwargs, color="black")
        ax.set(xlabel="Episode", ylabel="(%)")
        df['smooth_normalized_nnz'] = smooth(df['normalized_nnz'], window_size)
        df['smooth_normalized_nnz'][start:end].plot(**smooth_kwargs, color="black")        

    if reward:
        ax = df['reward'][start:end].plot(**plot_kwargs, secondary_y=True, color="g")
        ax.set(xlabel="Episode", ylabel="Reward")
        df['smooth_reward'] = smooth(df['reward'], window_size)
        df['smooth_reward'][start:end].plot(**smooth_kwargs, secondary_y=True, color="g")    
    #ax.set_ylim([0,100])
    #ax.grid(True, which='minor', axis='x', alpha=0.3)
    plt.xlabel("Episode")
    
    # The left axis might contain multiple ylabels
    if left_axs:
        # Pick an arbitrary axis 
        ax = left_axs[0][0]
        # Collect all of the labels
        ylabel = " / ".join([ax[1] for ax in left_axs]) #left_axs[0][1]
        ax.set(ylabel=ylabel)


        
def plot_2d_embeddings(top1, normalized_macs):
    plt.figure(figsize=(15,7))        
    plt.title('Projection of Discovered Networks ({})'.format(len(top1)))     
    plt.xlabel('Normalized MACs')
    plt.ylabel('Top1 Accuracy')

    # Create the formatter using the function to_percent. This multiplies all the
    # default labels by 100, making them all percentages
    formatter = FuncFormatter(to_percent)

    # Set the formatter
    plt.gca().yaxis.set_major_formatter(formatter)
    plt.gca().xaxis.set_major_formatter(formatter)

    # Use color gradients to show the "age" of the network:
    # Lighter networks were discovered earlier than darker ones.
    color_grad = [str(1-i/len(top1)) for i in range(len(top1))]
    plt.scatter(normalized_macs, top1, color=color_grad, s=80, edgecolors='gray');

    
INTERVAL = 30 # Animation speed
WINDOW = 20

font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'alpha': 0.50,
        'size': 32,
        }

# Based on these two helpful example code: 
# https://stackoverflow.com/questions/9401658/how-to-animate-a-scatter-plot
# http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-notebooks/.
# Specifically, the use of IPython.display is missing from the first example, but most of the animation code
# leverages code from there.
class AnimatedScatter(object):
    """An animated scatter plot using matplotlib.animations.FuncAnimation."""
    def __init__(self, xdata, ydata):
        assert len(xdata) == len(ydata)
        self.numpoints = len(xdata)
        self.xdata = xdata
        self.ydata = ydata
        self.stream = self.data_stream()

        # Setup the figure and axes...
        self.fig, self.ax = plt.subplots(figsize=(15,7))
        # Then setup FuncAnimation.
        self.ani = animation.FuncAnimation(self.fig, self.update, interval=INTERVAL,
                                           frames=self.numpoints-2, 
                                           init_func=self.setup_plot, blit=True)

    def setup_plot(self):
        """Initialize drawing of the scatter plot."""
        x, y, s, c = next(self.stream)
        #self.annot = self.ax.annotate("txt", (10, 10))
        self.scat = self.ax.scatter(x, y, c=c, s=s, animated=False)
        self.scat.set_edgecolors('gray')
        self.scat.set_cmap('gray')
        self.width = max(self.xdata) - min(self.xdata) + 4
        self.height = max(self.ydata) - min(self.ydata) + 4
        self.ax.axis([min(self.xdata)-2, max(self.xdata)+2, 
                      min(self.ydata)-2, max(self.ydata)+2])
        
        self.annot = self.ax.text(min(self.xdata) + self.width/2, 
                     min(self.xdata) + self.height/2, 
                     "", fontdict=font)
        # For FuncAnimation's sake, we need to return the artist we'll be using
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat, 

    def data_stream(self):
        numpoints = 0#len(self.xdata)
        colors = []
        xxx = 0
        while True:
            numpoints += 1
            win_len = min(WINDOW, numpoints)
            data = np.ndarray((4, win_len))
            start = max(0,numpoints-WINDOW-1)
            data[0, :] = self.xdata[start:start+win_len]
            data[1, :] = self.ydata[start:start+win_len]
            data[2, :] = [70] * win_len  # point size
            #data[3, :] = [np.random.random() for p in range(numpoints)]  # color
            # The color of the points is a gradient with larger values for "younger" points.
            # At each new frame we show one more point, and "age" each existing point by incrementaly  
            # reducing its color gradient.
            data[3, :] = [(1-i/(win_len+1)) for i in range(win_len)] 
            yield data

    def update(self, i):      
        """Update the scatter plot."""
        data = next(self.stream)
        self.annot.set_text(i)
        i = i % len(data)
            
        # Set x and y data
        xy = [(data[0,i], data[1,i]) for i in range(len(data[0,:]))]
        self.scat.set_offsets(xy)
        
        # Set colors
        self.scat.set_array(data[3])
        
        # We need to return the updated artist for FuncAnimation to draw..
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat, self.annot

    def show(self):
        plt.show()


## Results

Below I present the results of a single execution.  There is a substantial variance between the experiment executions, but most conclude similarly to this experiment.

### Read the results log files

The code below reads the log file of your selected experiment.  To change the path to the file you will need to open the code cell and change its content.

In [ ]:
pd.set_option('display.max_colwidth', 150)
fpath = "../classifier_compression/logs/{}/amc.csv"

fname1= fpath.format("mobilenet_v1_71.2_private_l2-norm_50-pts___2019.07.04-000305/")

df = pd.read_csv(fname)
df1, df2, df3 = pd.read_csv(fname1), pd.read_csv(fname2), pd.read_csv(fname3)
df4 = pd.read_csv(fname4)
#df[600:]

print(len(df))

### Plot experiment performance

In [ ]:
#plt.figure(figsize=(15,7))
#print(plt.style.available)
#plt.style.use('seaborn-paper')
        
@interact(window_size=(0,50,5), top1=True, macs=True, params=False, reward=True)
def plot_performance_proxy(window_size=10, top1=True, macs=True, params=False, reward=True):
    plot_performance("Training Data", df, 0.15, window_size, top1, macs, params, reward)

### Compare experiments

In [ ]:
dfs = [df1,df2, df3]
df_len = min([len(df) for df in dfs])

@interact(window_size=(0,50,5), top1=True, macs=True, params=False, reward=True, zoom=(0,df_len,1))
def plot_experiment_comparison(window_size=10, zoom=0):
    start = 0
    end = zoom if zoom > 0 else 0
    plot_performance("Compare Experiments (Top1)", dfs, 
                     0.15, window_size, True, False, False, False, start, end, plot_type='compare')

In [ ]:
@interact(window_size=(0,50,5), top1=True, macs=True, params=False, reward=True)
def plot_experiment_comparison(window_size=10):
    dfs = [df1,df2, df3]
    plot_performance("Compare Experiments (Top1)", dfs, 
                     0.15, window_size, True, False, False, False, plot_type='error')

### Sample some networks

Let's look at the networks with the best top1 accuracy, and see if they share geometrical attributes.

We sort the discovered networks by their Top1 accuracy and display the density of each layer in the networks.  

In [ ]:
top1_sorted_df = df1.sort_values(by=['reward'], ascending=False)
top1_sorted_df
nrows = 2; ncols = 4
f, axarr = plt.subplots(nrows, ncols, figsize=(15,7))
for i in range(0, nrows * ncols):
    plot_layer_compute_densities(top1_sorted_df, i, ax=axarr[i//ncols, i%ncols], color='g')
    # Fine-tune figure; make subplots farther from each other.
    f.subplots_adjust(hspace=0.6, wspace=0.4)

#### These are the actions the networks experienced

In [ ]:
#top1_sorted_df = df.sort_values(by=['reward'], ascending=False)
nrows = 2; ncols = 4
f, axarr = plt.subplots(nrows, ncols, figsize=(15,7))
for i in range(0, nrows * ncols):
    plot_action_history(top1_sorted_df, i, ax=axarr[i//ncols, i%ncols], color='g')
    # Fine-tune figure; make subplots farther from each other.
    f.subplots_adjust(hspace=0.6, wspace=0.4)

### Per-layer filter density distribution - top 10% networks

In [ ]:
top10pct = top1_sorted_df[:int(len(df.index) * 0.1)]
#top10pct = df[int(len(df.index) * 0.95):]

layer_densities_list = []
for index, row in top10pct.iterrows():
    #layer_sparsities = row['action_history']
    layer_sparsities = json.loads(row['action_history'])
    #layer_sparsities = layer_sparsities[1:-1].split(",")  # convert from string to list
    layer_densities = [1. - float(sparsity) for sparsity in layer_sparsities]
    layer_densities_list.append(layer_densities)

layer_densities = np.array(layer_densities_list)
mean = layer_densities.mean(axis=0)
std = layer_densities.std(axis=0)


# Draw the bar diagram of the layer densities
fig, ax = plt.subplots(figsize=(15,7.5))
ax.set_title("Best sampled models (90th percentile)\nPer-layer filter density (mean and std)")
xpos = [i for i in range(len(mean))]
ax.bar(xpos, mean, yerr=std, capsize=10, alpha=0.5, ecolor='black')
ax.set_ylabel('Filter density')
ax.set_xticks(xpos)
#ax.set_xticklabels(layer_names)
ax.yaxis.grid(True)
#plt.tight_layout()
plt.show()

### Network 2D embeddings

Let's create an embedding of the networks AMC discovers over the course of each experiment session.  Each network is projected onto a 2D plane mapping the Top1 accuracy versus the compute budget, and is represented by a small circle. I used gradient-color-coding to show the relative phase where each network is discovered.  Lighter circles are networks discovered early in the search, darker networks are discovered later.

In [ ]:
top1 = df['top1']
normalized_macs = df['normalized_macs']
plot_2d_embeddings(top1, normalized_macs)

### Video animation

In [ ]:
a = AnimatedScatter(normalized_macs, top1)
plt.title('Projection of Discovered Networks ({})'.format(len(top1)))  
plt.xlabel('Normalized MACs')
plt.ylabel('Top1 Accuracy')
#a.ani.save('amc_vgg16.mp4', fps=10, dpi=80) #Frame per second controls speed, dpi controls the quality 
rc('animation', html='html5')
a.ani

## Plot a histogram of the actions

We want to look at the distribution of the sampled agent actions (```agent_actions``` below), to make sure they are not skewed or biased.

We also want to compare these actions to the actions the environment actually acts on (these we call ```env_actions``` below).  The natural PPO action-space is (-inf, inf) so the environment needs to scale and shift the agent actions to fit into the "real" action-space.

In [ ]:
def get_all_actions(df, action_type='action_history'):
    actions = []
    for index, record in df.iterrows():
        #layer_sparsities = record[action_type]
        #layer_sparsities = layer_sparsities[1:-1].split(",")
        layer_sparsities = json.loads(record[action_type])
        layer_sparsities = [float(sparsity) for sparsity in layer_sparsities]
        actions.extend(layer_sparsities)
    return actions

In [ ]:
agent_actions = get_all_actions(df, action_type='agent_action_history')
env_actions = get_all_actions(df, action_type='action_history')
print(len(agent_actions))

plt.figure(figsize=[15,7.5])
plt.hist(agent_actions, histtype='step', bins=100, label='agent_actions (u={:.2f} std={:.2f})'.format(
                                                                                        np.mean(agent_actions),
                                                                                        np.std(agent_actions)));
plt.hist(env_actions, histtype='step', bins=100, label='env_actions (u={:.2f} std={:.2f})'.format(
                                                                                        np.mean(env_actions),
                                                                                        np.std(env_actions)));

plt.title('Action histogram (actions as seen by the environment)')
plt.legend()
plt.xlabel('Action value')
plt.ylabel('Frequency');